### Data Preprocessing

In [1]:
import numpy as np 
from tqdm import tqdm
import cv2
import os
import imutils
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# 在数据加载前添加内存优化代码
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # 设置GPU内存按需增长
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

#### Cropping the images to remove background

In [3]:
def crop_img(img):
	"""
	Finds the extreme points on the image and crops the rectangular out of them
	"""
	gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
	gray = cv2.GaussianBlur(gray, (3, 3), 0)

	# threshold the image, then perform a series of erosions +
	# dilations to remove any small regions of noise
	thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
	thresh = cv2.erode(thresh, None, iterations=2)
	thresh = cv2.dilate(thresh, None, iterations=2)

	# find contours in thresholded image, then grab the largest one
	cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	cnts = imutils.grab_contours(cnts)
	c = max(cnts, key=cv2.contourArea)

	# find the extreme points
	extLeft = tuple(c[c[:, :, 0].argmin()][0])
	extRight = tuple(c[c[:, :, 0].argmax()][0])
	extTop = tuple(c[c[:, :, 1].argmin()][0])
	extBot = tuple(c[c[:, :, 1].argmax()][0])
	ADD_PIXELS = 0
	new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
	
	return new_img

##### Creating a cropped folder in which cropped training and testing images are stored

In [4]:
if __name__ == "__main__":
	training = "archive/Training"
	testing = "archive/Testing"
	training_dir = os.listdir(training)
	testing_dir = os.listdir(testing)
	IMG_SIZE = 256
	
	for dir in training_dir:
		save_path = 'Dataset/cropped/Training/'+ dir
		path = os.path.join(training,dir)
		image_dir = os.listdir(path)
		for img in image_dir:
			image = cv2.imread(os.path.join(path,img))
			new_img = crop_img(image)
			new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
			if not os.path.exists(save_path):
				os.makedirs(save_path)
			cv2.imwrite(save_path+'/'+img, new_img)
	
	for dir in testing_dir:
		save_path = 'Dataset/cropped/Testing/'+ dir
		path = os.path.join(testing,dir)
		image_dir = os.listdir(path)
		for img in image_dir:
			image = cv2.imread(os.path.join(path,img))
			new_img = crop_img(image)
			new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
			if not os.path.exists(save_path):
				os.makedirs(save_path)
			cv2.imwrite(save_path+'/'+img, new_img)


	

In [5]:
training_dir

['glioma', 'meningioma', 'notumor', 'pituitary']

##### Cropped images Demo 

In [6]:
# img = cv2.imread('./Dataset/Trainingbrain/meningioma/Tr-me_0476.jpg')
# img = cv2.resize(
#             img,
#             dsize=(224,224),
#             interpolation=cv2.INTER_CUBIC
#         )
# gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
# gray = cv2.GaussianBlur(gray, (5, 5), 0)

# # threshold the image, then perform a series of erosions +
# # dilations to remove any small regions of noise
# thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
# thresh = cv2.erode(thresh, None, iterations=2)
# thresh = cv2.dilate(thresh, None, iterations=2)

# # find contours in thresholded image, then grab the largest one
# cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cnts = imutils.grab_contours(cnts)
# c = max(cnts, key=cv2.contourArea)

# # find the extreme points
# extLeft = tuple(c[c[:, :, 0].argmin()][0])
# extRight = tuple(c[c[:, :, 0].argmax()][0])
# extTop = tuple(c[c[:, :, 1].argmin()][0])
# extBot = tuple(c[c[:, :, 1].argmax()][0])

# # add contour on the image
# img_cnt = cv2.drawContours(img.copy(), [c], -1, (0, 255, 255), 4)

# # add extreme points
# img_pnt = cv2.circle(img_cnt.copy(), extLeft, 8, (0, 0, 255), -1)
# img_pnt = cv2.circle(img_pnt, extRight, 8, (0, 255, 0), -1)
# img_pnt = cv2.circle(img_pnt, extTop, 8, (255, 0, 0), -1)
# img_pnt = cv2.circle(img_pnt, extBot, 8, (255, 255, 0), -1)

# # crop
# ADD_PIXELS = 0
# new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()

In [7]:
# plt.figure(figsize=(15,6))
# plt.subplot(141)
# plt.imshow(img)
# plt.xticks([])
# plt.yticks([])
# plt.title('Step 1. Get the original image')
# plt.subplot(142)
# plt.imshow(img_cnt)
# plt.xticks([])
# plt.yticks([])
# plt.title('Step 2. Find the biggest contour')
# plt.subplot(143)
# plt.imshow(img_pnt)
# plt.xticks([])
# plt.yticks([])
# plt.title('Step 3. Find the extreme points')
# plt.subplot(144)
# plt.imshow(new_img)
# plt.xticks([])
# plt.yticks([])
# plt.title('Step 4. Crop the image')
# plt.show()

##### Creating X and Y variables

In [8]:
X_train = []
y_train = []
X_test = []
y_test = []
image_size = 200
labels = ['glioma','meningioma','notumor','pituitary'] 
for i in labels:
    folderpath = os.path.join('./Dataset/Cropped/Training',i)
    for j in os.listdir(folderpath):
        image = cv2.imread(os.path.join(folderpath,j),0) #load images in gray
        image = cv2.bilateralFilter(image,2,50,50) # remove noise image
        image = cv2.applyColorMap(image, cv2.COLORMAP_BONE) # produce a pseudocolored image.
        image = cv2.resize(image, (image_size, image_size)) # resize images
        X_train.append(image)
        y_train.append(labels.index(i))
        
for i in labels:
    folderpath = os.path.join('./Dataset/Cropped/Testing',i)
    for j in os.listdir(folderpath):
        image = cv2.imread(os.path.join(folderpath,j),0) #load images in gray
        image = cv2.bilateralFilter(image,2,50,50) # remove noise image
        image = cv2.applyColorMap(image, cv2.COLORMAP_BONE) # produce a pseudocolored image.
        image = cv2.resize(image, (image_size, image_size)) # resize images
        X_test.append(image)
        y_test.append(labels.index(i))

X_train = np.array(X_train) / 255.0 # normalize Images into range 0 to 1.
X_test = np.array(X_test) / 255.0

print(X_train.shape)
print(X_test.shape)
        

(5712, 200, 200, 3)
(1311, 200, 200, 3)


(5712, 200, 200, 3) signifies the dimensions of the training dataset. 

- **5712**: This denotes the number of individual samples or images in the training set. Each sample serves as an instance for training the machine learning model.

- **200**: This indicates that each image has a width of 200 pixels. Resizing all images to this uniform width ensures compatibility and consistency when feeding them into a model.

- **200**: Similarly, this signifies the height of each image, also being 200 pixels. Having a consistent spatial size (width and height) is crucial for most machine learning and deep learning models.

- **3**: This represents the three color channels (Red, Green, Blue, or RGB) for each image. Despite initially reading the images in grayscale, the application of `cv2.applyColorMap` seems to have transformed them into color-mapped images with three channels, which is typical for colored images processed by models, especially convolutional neural networks (CNNs) that are designed to handle color information.

In summary, `X_train` is a four-dimensional array (or tensor) containing 5712 samples, each being a 200x200 pixel RGB image, well-suited for direct input into deep learning models for training.

In [9]:
# images = [X_train[i] for i in range(9)]
# fig, axes = plt.subplots(3, 3, figsize = (10, 10))
# axes = axes.flatten()
# for img, ax in zip(images, axes):
#     ax.imshow(img)
# #plt.tight_layout()
# plt.show()

### Train test split

In [10]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [11]:
X_train, y_train = shuffle(X_train,y_train, random_state=42) 

In [12]:
#One Hot Encoding on the labels
y_train = tf.keras.utils.to_categorical(y_train) 
y_test = tf.keras.utils.to_categorical(y_test)

Synthetic Data Preprocessing

In [13]:
import numpy as np
import cv2
import os
import imutils
import tensorflow as tf
from sklearn.utils import shuffle

# 1. Define the crop_img function (code1)
def crop_img(img):
    """
    Finds the extreme points on the image and crops the rectangular out of them
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    # threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)

    # find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    ADD_PIXELS = 0
    new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
    
    return new_img

# 2. Process synthetic dataset similar to code2
def process_synthetic_dataset():
    synthetic_source = "archive/synthetic dataset300"
    synthetic_dest = "Dataset/cropped/synthetic dataset300"
    IMG_SIZE = 256
    
    # Create destination directory if it doesn't exist
    os.makedirs(synthetic_dest, exist_ok=True)
    
    # Process each subfolder in synthetic dataset
    for class_folder in os.listdir(synthetic_source):
        source_path = os.path.join(synthetic_source, class_folder)
        dest_path = os.path.join(synthetic_dest, class_folder)
        
        # Skip if not a directory
        if not os.path.isdir(source_path):
            continue
            
        # Create destination subfolder
        os.makedirs(dest_path, exist_ok=True)
        
        # Process each image in the subfolder
        for img_file in os.listdir(source_path):
            img_path = os.path.join(source_path, img_file)
            
            try:
                # Read, crop and resize image
                image = cv2.imread(img_path)
                if image is None:
                    continue
                    
                new_img = crop_img(image)
                new_img = cv2.resize(new_img, (IMG_SIZE, IMG_SIZE))
                
                # Save processed image
                cv2.imwrite(os.path.join(dest_path, img_file), new_img)
                
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
                continue

# 3. Create X_synthetic and y_synthetic similar to code3
def create_synthetic_data_arrays():
    synthetic_path = "Dataset/cropped/synthetic dataset300"
    image_size = 200
    labels = ['glioma', 'meningioma', 'notumor', 'pituitary']
    
    X_synthetic = []
    y_synthetic = []
    
    for i, label in enumerate(labels):
        folderpath = os.path.join(synthetic_path, label)
        
        # Skip if folder doesn't exist
        if not os.path.exists(folderpath):
            continue
            
        for img_file in os.listdir(folderpath):
            img_path = os.path.join(folderpath, img_file)
            
            try:
                # Load and preprocess image
                image = cv2.imread(img_path, 0)  # load images in gray
                image = cv2.bilateralFilter(image, 2, 50, 50)  # remove noise
                image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)  # pseudocolor
                image = cv2.resize(image, (image_size, image_size))  # resize
                
                X_synthetic.append(image)
                y_synthetic.append(i)
                
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
                continue
    
    # Convert to numpy arrays and normalize
    X_synthetic = np.array(X_synthetic) / 255.0
    y_synthetic = np.array(y_synthetic)
    
    print(f"Synthetic data shape: {X_synthetic.shape}")
    return X_synthetic, y_synthetic

# 4. Shuffle and one-hot encode similar to code4
def preprocess_labels(X, y):
    # Shuffle the data
    X, y = shuffle(X, y, random_state=42)
    
    # One-hot encode the labels
    y = tf.keras.utils.to_categorical(y, num_classes=4)
    
    return X, y

# Main execution
if __name__ == "__main__":
    print("Step 1: Processing synthetic dataset (cropping and resizing)...")
    process_synthetic_dataset()
    
    print("\nStep 2: Creating X_synthetic and y_synthetic arrays...")
    X_synthetic, y_synthetic = create_synthetic_data_arrays()
    
    print("\nStep 3: Final preprocessing (shuffling and one-hot encoding)...")
    X_synthetic, y_synthetic = preprocess_labels(X_synthetic, y_synthetic)
    
    print("\nPreprocessing completed!")
    print(f"Final X_synthetic shape: {X_synthetic.shape}")
    print(f"Final y_synthetic shape: {y_synthetic.shape}")
    

Step 1: Processing synthetic dataset (cropping and resizing)...

Step 2: Creating X_synthetic and y_synthetic arrays...
Synthetic data shape: (1140, 200, 200, 3)

Step 3: Final preprocessing (shuffling and one-hot encoding)...

Preprocessing completed!
Final X_synthetic shape: (1140, 200, 200, 3)
Final y_synthetic shape: (1140, 4)


In [14]:
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42) #Dividing the dataset into Training and Validation sets.
# print(X_val.shape)
# print(X_train.shape)
# print(X_test.shape)

### Image Augmentation

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Dropout,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [16]:
# 修改数据生成器设置
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True,
    dtype='float16'  # 使用16位浮点数减少内存占用
)

# After you have created and configured your ImageDataGenerator, you must fit it on your data.
datagen.fit(X_train)
datagen.fit(X_synthetic)

### Model Training

In [17]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from sklearn.metrics import classification_report, confusion_matrix
# import pandas as pd
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.model_selection import train_test_split

# # Federated Learning Parameters
# NUM_CLASSES = 4  # glioma, meningioma, notumor, pituitary
# CLIENTS_PER_CLASS = 10  # 5 clients per class
# NUM_CLIENTS = NUM_CLASSES * CLIENTS_PER_CLASS  # 20 total clients
# NUM_ROUNDS = 50
# CLIENT_FRACTION = 0.5  # Fraction of clients participating each round
# SYNTHETIC_CLIENT_RATIO = 0.5  # Each client gets 50% of synthetic data

# # Early Stopping Parameters
# WINDOW_SIZE = 5  # Larger window for non-IID data
# MIN_DELTA = 0.001  # Smaller delta for non-IID
# PATIENCE = 5  # More patience for non-IID

# # Class names for reporting
# class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

# def save_classification_report(y_true, y_pred, round_num, save_dir):
#     report = classification_report(y_true, y_pred, target_names=class_names)
#     with open(f"{save_dir}/classification_report_round_{round_num}.txt", "w") as f:
#         f.write(report)
#     return report

# def save_confusion_matrix(y_true, y_pred, round_num, save_dir):
#     cm = confusion_matrix(y_true, y_pred)
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
#                 xticklabels=class_names, yticklabels=class_names)
#     plt.title(f'Confusion Matrix (Round {round_num})')
#     plt.ylabel('True Label')
#     plt.xlabel('Predicted Label')
#     plt.savefig(f"{save_dir}/confusion_matrix_round_{round_num}.pdf")
#     plt.close()
#     return cm

# def create_global_model():
#     IMG_SIZE = (200, 200)
#     base_model = VGG19(
#         include_top=False,
#         input_shape=IMG_SIZE + (3,),
#         weights='imagenet')

#     for layer in base_model.layers:
#         layer.trainable = False
#     for layer in base_model.layers[-7:]:
#         layer.trainable = True

#     x = GlobalAveragePooling2D()(base_model.output)
#     x = Dropout(0.4)(x)
#     predict = Dense(NUM_CLASSES, activation='softmax')(x)

#     model = Model(inputs=base_model.input, outputs=predict)
    
#     optimizer = Adam(learning_rate=0.0001)
#     model.compile(optimizer=optimizer, 
#                  loss='categorical_crossentropy', 
#                  metrics=['accuracy'])
    
#     return model

# def initialize_with_synthetic(X_synthetic, y_synthetic):
#     """Train initial model on synthetic data without validation split"""
#     print("\nInitializing model with synthetic data (no validation split)...")
#     model = create_global_model()
    
#     # Train on full synthetic dataset without validation split
#     model.fit(
#         datagen.flow(X_synthetic, y_synthetic, batch_size=16),
#         epochs=10,
#         verbose=1)
#     return model

# def create_client_data(X_client, y_client, X_synthetic, y_synthetic):
#     """Combine client-specific data with shared synthetic data"""
#     # Get 50% of synthetic data (IID)
#     idx = np.random.choice(len(X_synthetic), 
#                          size=int(len(X_synthetic)*SYNTHETIC_CLIENT_RATIO), 
#                          replace=False)
#     client_synthetic_X = X_synthetic[idx]
#     client_synthetic_y = y_synthetic[idx]
    
#     # Combine with client's non-IID data
#     combined_X = np.concatenate([X_client, client_synthetic_X])
#     combined_y = np.concatenate([y_client, client_synthetic_y])
    
#     # Shuffle the combined data
#     p = np.random.permutation(len(combined_X))
#     return combined_X[p], combined_y[p]

# # Data Preparation
# # Use all training data for clients (no holdout)
# X_client = X_train
# y_client = y_train

# # Create non-IID clients from all training data
# def create_non_iid_clients(X, y):
#     y_labels = np.argmax(y, axis=1)
#     class_data = {}
#     for class_id in range(NUM_CLASSES):
#         class_indices = np.where(y_labels == class_id)[0]
#         class_data[class_id] = X[class_indices], y[class_indices]
    
#     clients = []
#     for class_id in range(NUM_CLASSES):
#         X_class, y_class = class_data[class_id]
#         samples_per_client = len(X_class) // CLIENTS_PER_CLASS
        
#         for client_id in range(CLIENTS_PER_CLASS):
#             start = client_id * samples_per_client
#             end = (client_id + 1) * samples_per_client
#             clients.append((X_class[start:end], y_class[start:end]))
#     return clients

# # Create base clients with only non-IID data
# base_clients = create_non_iid_clients(X_client, y_client)

# # Create final clients by combining with synthetic data
# clients = [create_client_data(X, y, X_synthetic, y_synthetic) for X, y in base_clients]

# # # Data augmentation
# # datagen = ImageDataGenerator(
# #     rotation_range=10,
# #     width_shift_range=0.05,
# #     height_shift_range=0.05,
# #     horizontal_flip=True)

# # Create results directory
# os.makedirs('results_noHoldout', exist_ok=True)
# os.makedirs('results_noHoldout/reports', exist_ok=True)
# os.makedirs('results_noHoldout/confusion_matrices', exist_ok=True)

# # Initialize model on synthetic data
# global_model = initialize_with_synthetic(X_synthetic, y_synthetic)

# # Evaluate and save initialization model performance (Round 0)
# print("\nEvaluating initialization model (Round 0)...")
# y_pred = global_model.predict(X_test, verbose=0)
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_true = np.argmax(y_test, axis=1)

# # Save initialization model reports
# save_classification_report(y_true, y_pred_classes, 0, 'results_noHoldout/reports')
# save_confusion_matrix(y_true, y_pred_classes, 0, 'results_noHoldout/confusion_matrices')

# initial_loss, initial_accuracy = global_model.evaluate(X_test, y_test, verbose=0)
# print(f"Initial model accuracy: {initial_accuracy:.4f}, Loss: {initial_loss:.4f}")

# # Initialize tracking variables with initial model performance
# results = [{
#     'round': 0,
#     'accuracy': initial_accuracy,
#     'loss': initial_loss
# }]

# best_accuracy = initial_accuracy
# best_model_weights = global_model.get_weights()
# recent_accuracies = [initial_accuracy]
# no_improvement_count = 0
# early_stop_triggered = False 

# # Federated Learning Main Loop
# for round_num in range(NUM_ROUNDS):
#     if early_stop_triggered:
#         print("Early stopping triggered. Ending training.")
#         break
        
#     print(f"\n=== Federated Learning Round {round_num + 1}/{NUM_ROUNDS} ===")
    
#     # Select clients
#     selected_indices = []
#     for class_id in range(NUM_CLASSES):
#         class_client_indices = range(class_id*CLIENTS_PER_CLASS, (class_id+1)*CLIENTS_PER_CLASS)
#         selected = np.random.choice(
#             list(class_client_indices),
#             size=max(1, int(CLIENT_FRACTION * CLIENTS_PER_CLASS)),
#             replace=False)
#         selected_indices.extend(selected)
    
#     selected_clients = [clients[i] for i in selected_indices]
    
#     # Local training
#     client_weights = []
#     client_sizes = []
    
#     for client_idx, (client_X, client_y) in enumerate(selected_clients):
#         print(f"\nTraining Client {client_idx + 1}/{len(selected_clients)}...")
        
#         client_model = tf.keras.models.clone_model(global_model)
#         client_model.set_weights(global_model.get_weights())
#         client_model.compile(optimizer=Adam(learning_rate=0.0001),
#                            loss='categorical_crossentropy',
#                            metrics=['accuracy'])
        
#         history = client_model.fit(
#             datagen.flow(client_X, client_y, batch_size=16),
#             epochs=5,
#             verbose=1)
        
#         client_weights.append(client_model.get_weights())
#         client_sizes.append(len(client_X))
    
#     # Federated Averaging
#     global_weights = []
#     total_size = sum(client_sizes)
    
#     for layer_idx in range(len(global_model.get_weights())):
#         weighted_sum = np.zeros_like(global_model.get_weights()[layer_idx])
#         for client_idx in range(len(client_weights)):
#             weighted_sum += client_weights[client_idx][layer_idx] * (client_sizes[client_idx] / total_size)
#         global_weights.append(weighted_sum)
    
#     global_model.set_weights(global_weights)
    
#     # Evaluation
#     y_pred = global_model.predict(X_test, verbose=0)
#     y_pred_classes = np.argmax(y_pred, axis=1)
#     y_true = np.argmax(y_test, axis=1)
    
#     save_classification_report(y_true, y_pred_classes, round_num+1, 'results_noHoldout/reports')
#     save_confusion_matrix(y_true, y_pred_classes, round_num+1, 'results_noHoldout/confusion_matrices')
    
#     loss, accuracy = global_model.evaluate(X_test, y_test, verbose=0)
#     print(f"Test Accuracy: {accuracy:.4f}, Loss: {loss:.4f}")
    
#     results.append({
#         'round': round_num + 1,
#         'accuracy': accuracy,
#         'loss': loss
#     })
    
#     if accuracy > best_accuracy + MIN_DELTA:
#         best_accuracy = accuracy
#         best_model_weights = global_model.get_weights()
#         no_improvement_count = 0
#         print(f"New best accuracy: {best_accuracy:.4f}")
#     else:
#         no_improvement_count += 1
    
#     recent_accuracies.append(accuracy)
#     if len(recent_accuracies) >= WINDOW_SIZE:
#         window_avg = np.mean(recent_accuracies[-WINDOW_SIZE:])
#         prev_window_avg = np.mean(recent_accuracies[-WINDOW_SIZE-1:-1] if len(recent_accuracies) > WINDOW_SIZE else recent_accuracies[:-1])
        
#         if window_avg <= prev_window_avg + MIN_DELTA:
#             no_improvement_count += 1
#             print(f"No improvement count: {no_improvement_count}/{PATIENCE}")
#         else:
#             no_improvement_count = max(0, no_improvement_count - 1)
        
#         if no_improvement_count >= PATIENCE:
#             early_stop_triggered = True

# # Save results and models
# df_results = pd.DataFrame(results)
# df_results.to_csv('results_noHoldout/training_results.csv', index=False)

# global_model.set_weights(best_model_weights)
# global_model.save('results_noHoldout/best_global_model.h5')

# # Final evaluation
# y_pred = global_model.predict(X_test, verbose=0)
# y_pred_classes = np.argmax(y_pred, axis=1)
# save_classification_report(y_true, y_pred_classes, 'final', 'results_noHoldout/reports')
# save_confusion_matrix(y_true, y_pred_classes, 'final', 'results_noHoldout/confusion_matrices')

# global_model.save('results_noHoldout/final_global_model.h5')
# print("\nFederated learning completed.")

In [18]:

import gc  # 添加垃圾回收模块

import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Federated Learning Parameters
NUM_CLASSES = 4  # glioma, meningioma, notumor, pituitary
CLIENTS_PER_CLASS = 8  # 5 clients per class
NUM_CLIENTS = NUM_CLASSES * CLIENTS_PER_CLASS  # 20 total clients
NUM_ROUNDS = 50
CLIENT_FRACTION = 0.5  # Fraction of clients participating each round
SYNTHETIC_CLIENT_RATIO = 0.5  # Each client gets 50% of synthetic data

# Early Stopping Parameters
WINDOW_SIZE = 5  # Larger window for non-IID data
MIN_DELTA = 0.001  # Smaller delta for non-IID
PATIENCE = 5  # More patience for non-IID

# Class names for reporting
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

def save_classification_report(y_true, y_pred, round_num, save_dir):
    report = classification_report(y_true, y_pred, target_names=class_names)
    with open(f"{save_dir}/classification_report_round_{round_num}.txt", "w") as f:
        f.write(report)
    return report

def save_confusion_matrix(y_true, y_pred, round_num, save_dir):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix (Round {round_num})')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(f"{save_dir}/confusion_matrix_round_{round_num}.pdf")
    plt.close()
    return cm

def create_global_model():
    IMG_SIZE = (200, 200)
    base_model = VGG19(
        include_top=False,
        input_shape=IMG_SIZE + (3,),
        weights='imagenet')

    for layer in base_model.layers:
        layer.trainable = False
    for layer in base_model.layers[-7:]:
        layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.4)(x)
    predict = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predict)
    
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, 
                 loss='categorical_crossentropy', 
                 metrics=['accuracy'])
    
    return model

def initialize_with_synthetic(X_synthetic, y_synthetic):
    """Train initial model on synthetic data without validation split"""
    print("\nInitializing model with synthetic data (no validation split)...")
    model = create_global_model()
    
    # Train on full synthetic dataset without validation split
    model.fit(
        datagen.flow(X_synthetic, y_synthetic, batch_size=16),
        epochs=10,
        verbose=1)
    return model

def create_client_data(X_client, y_client, X_synthetic, y_synthetic):
    """Combine client-specific data with shared synthetic data"""
    # Get 50% of synthetic data (IID)
    idx = np.random.choice(len(X_synthetic), 
                         size=int(len(X_synthetic)*SYNTHETIC_CLIENT_RATIO), 
                         replace=False)
    client_synthetic_X = X_synthetic[idx]
    client_synthetic_y = y_synthetic[idx]
    
    # Combine with client's non-IID data
    combined_X = np.concatenate([X_client, client_synthetic_X])
    combined_y = np.concatenate([y_client, client_synthetic_y])
    
    # Shuffle the combined data
    p = np.random.permutation(len(combined_X))
    return combined_X[p], combined_y[p]

# Data Preparation
# Use all training data for clients (no holdout)
X_client = X_train
y_client = y_train

# Create non-IID clients from all training data
def create_non_iid_clients(X, y):
    y_labels = np.argmax(y, axis=1)
    class_data = {}
    for class_id in range(NUM_CLASSES):
        class_indices = np.where(y_labels == class_id)[0]
        class_data[class_id] = X[class_indices], y[class_indices]
    
    clients = []
    for class_id in range(NUM_CLASSES):
        X_class, y_class = class_data[class_id]
        samples_per_client = len(X_class) // CLIENTS_PER_CLASS
        
        for client_id in range(CLIENTS_PER_CLASS):
            start = client_id * samples_per_client
            end = (client_id + 1) * samples_per_client
            clients.append((X_class[start:end], y_class[start:end]))
    return clients

# Create base clients with only non-IID data
base_clients = create_non_iid_clients(X_client, y_client)

# Create final clients by combining with synthetic data
clients = [create_client_data(X, y, X_synthetic, y_synthetic) for X, y in base_clients]

# # Data augmentation
# datagen = ImageDataGenerator(
#     rotation_range=10,
#     width_shift_range=0.05,
#     height_shift_range=0.05,
#     horizontal_flip=True)

# Create results directory
os.makedirs('results_noHoldout', exist_ok=True)
os.makedirs('results_noHoldout/reports', exist_ok=True)
os.makedirs('results_noHoldout/confusion_matrices', exist_ok=True)

# Initialize model on synthetic data
global_model = initialize_with_synthetic(X_synthetic, y_synthetic)

# Evaluate and save initialization model performance (Round 0)
print("\nEvaluating initialization model (Round 0)...")
y_pred = global_model.predict(X_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Save initialization model reports
save_classification_report(y_true, y_pred_classes, 0, 'results_noHoldout/reports')
save_confusion_matrix(y_true, y_pred_classes, 0, 'results_noHoldout/confusion_matrices')

initial_loss, initial_accuracy = global_model.evaluate(X_test, y_test, verbose=0)
print(f"Initial model accuracy: {initial_accuracy:.4f}, Loss: {initial_loss:.4f}")

# Initialize tracking variables with initial model performance
results = [{
    'round': 0,
    'accuracy': initial_accuracy,
    'loss': initial_loss
}]

best_accuracy = initial_accuracy
best_model_weights = global_model.get_weights()
recent_accuracies = [initial_accuracy]
no_improvement_count = 0
early_stop_triggered = False 


# Federated Learning Main Loop
for round_num in range(NUM_ROUNDS):
    if early_stop_triggered:
        print("Early stopping triggered. Ending training.")
        break
        
    print(f"\n=== Federated Learning Round {round_num + 1}/{NUM_ROUNDS} ===")
    
    # Select clients (保持不变)
    selected_indices = []
    for class_id in range(NUM_CLASSES):
        class_client_indices = range(class_id*CLIENTS_PER_CLASS, (class_id+1)*CLIENTS_PER_CLASS)
        selected = np.random.choice(
            list(class_client_indices),
            size=max(1, int(CLIENT_FRACTION * CLIENTS_PER_CLASS)),
            replace=False)
        selected_indices.extend(selected)
    
    selected_clients = [clients[i] for i in selected_indices]
    
    # 修改后的联邦平均逻辑
    global_weights = None
    total_size = 0
    
    for client_idx, (client_X, client_y) in enumerate(selected_clients):
        print(f"\nTraining Client {client_idx + 1}/{len(selected_clients)}...")
        
        # 1. 创建并训练客户端模型
        client_model = tf.keras.models.clone_model(global_model)
        client_model.set_weights(global_model.get_weights())
        client_model.compile(
            optimizer=Adam(learning_rate=0.0001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        history = client_model.fit(
            datagen.flow(client_X, client_y, batch_size=8),
            epochs=5,
            verbose=1
        )
        
        # 2. 逐步累加权重（避免存储全部客户端权重）
        client_size = len(client_X)
        if global_weights is None:
            global_weights = [w * client_size for w in client_model.get_weights()]
        else:
            for i in range(len(global_weights)):
                global_weights[i] += client_model.get_weights()[i] * client_size
        total_size += client_size
        
        # 3. 显式释放资源
        del client_model
        gc.collect()  # 强制垃圾回收
    
    # 4. 计算全局权重
    if global_weights is not None:
        global_weights = [w / total_size for w in global_weights]
        global_model.set_weights(global_weights)
    
    # Evaluation
    y_pred = global_model.predict(X_test, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)
    
    save_classification_report(y_true, y_pred_classes, round_num+1, 'results_noHoldout/reports')
    save_confusion_matrix(y_true, y_pred_classes, round_num+1, 'results_noHoldout/confusion_matrices')
    
    loss, accuracy = global_model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {accuracy:.4f}, Loss: {loss:.4f}")
    
    results.append({
        'round': round_num + 1,
        'accuracy': accuracy,
        'loss': loss
    })
    
    if accuracy > best_accuracy + MIN_DELTA:
        best_accuracy = accuracy
        best_model_weights = global_model.get_weights()
        no_improvement_count = 0
        print(f"New best accuracy: {best_accuracy:.4f}")
    else:
        no_improvement_count += 1
    
    recent_accuracies.append(accuracy)
    if len(recent_accuracies) >= WINDOW_SIZE:
        window_avg = np.mean(recent_accuracies[-WINDOW_SIZE:])
        prev_window_avg = np.mean(recent_accuracies[-WINDOW_SIZE-1:-1] if len(recent_accuracies) > WINDOW_SIZE else recent_accuracies[:-1])
        
        if window_avg <= prev_window_avg + MIN_DELTA:
            no_improvement_count += 1
            print(f"No improvement count: {no_improvement_count}/{PATIENCE}")
        else:
            no_improvement_count = max(0, no_improvement_count - 1)
        
        if no_improvement_count >= PATIENCE:
            early_stop_triggered = True

# Save results and models
df_results = pd.DataFrame(results)
df_results.to_csv('results_noHoldout/training_results.csv', index=False)

global_model.set_weights(best_model_weights)
global_model.save('results_noHoldout/best_global_model.h5')

# Final evaluation
y_pred = global_model.predict(X_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
save_classification_report(y_true, y_pred_classes, 'final', 'results_noHoldout/reports')
save_confusion_matrix(y_true, y_pred_classes, 'final', 'results_noHoldout/confusion_matrices')

global_model.save('results_noHoldout/final_global_model.h5')
print("\nFederated learning completed.")


Initializing model with synthetic data (no validation split)...
Epoch 1/10
72/72 [==============================] - 11s 66ms/step - loss: 0.6030 - accuracy: 0.7456
Epoch 2/10
72/72 [==============================] - 4s 59ms/step - loss: 0.1488 - accuracy: 0.9456
Epoch 3/10
72/72 [==============================] - 4s 59ms/step - loss: 0.0885 - accuracy: 0.9711
Epoch 4/10
72/72 [==============================] - 4s 58ms/step - loss: 0.0683 - accuracy: 0.9746
Epoch 5/10
72/72 [==============================] - 4s 58ms/step - loss: 0.0324 - accuracy: 0.9886
Epoch 6/10
72/72 [==============================] - 4s 58ms/step - loss: 0.0306 - accuracy: 0.9895
Epoch 7/10
72/72 [==============================] - 4s 57ms/step - loss: 0.0298 - accuracy: 0.9912
Epoch 8/10
72/72 [==============================] - 4s 57ms/step - loss: 0.0237 - accuracy: 0.9912
Epoch 9/10
72/72 [==============================] - 4s 58ms/step - loss: 0.0970 - accuracy: 0.9675
Epoch 10/10
72/72 [========================

MemoryError: Unable to allocate 337. MiB for an array with shape (737, 200, 200, 3) and data type float32

In [ ]:
# from tensorflow.keras.applications.vgg19 import VGG19
# IMG_SIZE=(200,200)
# base_model = VGG19(
#     include_top=False,
#     input_shape=IMG_SIZE + (3,),
#     weights='imagenet')

# for layer in base_model.layers:
#     layer.trainable = True

# # Customized layers
# x = GlobalAveragePooling2D()(base_model.output)
# x = Dropout(0.4)(x)
# predict = Dense(4,activation='softmax')(x)

# # create a model object
# model = Model(inputs = base_model.input,outputs = predict)

# #compile our model.
# adam = Adam(learning_rate=0.0001)
# model.compile(optimizer=adam, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from sklearn.model_selection import train_test_split

# # Federated Learning Parameters
# NUM_CLIENTS = 15
# NUM_ROUNDS = 2
# CLIENT_FRACTION = 0.5  # Fraction of clients participating in each round

# # Data Preparation (Assume X_train, y_train are prepared as per previous code)
# # X_train shape: (n_samples, 200, 200, 3)
# # y_train shape: (n_samples, 4) - one-hot encoded

# # Split data among clients
# def create_clients(X_train, y_train, num_clients=NUM_CLIENTS):
#     clients = []
#     data_per_client = len(X_train) // num_clients
    
#     for i in range(num_clients):
#         client_X = X_train[i*data_per_client : (i+1)*data_per_client]
#         client_y = y_train[i*data_per_client : (i+1)*data_per_client]
#         clients.append((client_X, client_y))
    
#     return clients

# clients = create_clients(X_train, y_train)

# # Initialize the global model
# def create_global_model():
#     IMG_SIZE = (200, 200)
#     base_model = VGG19(
#         include_top=False,
#         input_shape=IMG_SIZE + (3,),
#         weights='imagenet')

#     # Default: Freeze all layers
#     for layer in base_model.layers:
#         layer.trainable = False
#     for layer in base_model.layers[-7:]:
#         layer.trainable = True

#     # Custom classification head
#     x = GlobalAveragePooling2D()(base_model.output)
#     x = Dropout(0.4)(x)
#     predict = Dense(4, activation='softmax')(x)

#     # Create the model
#     model = Model(inputs=base_model.input, outputs=predict)
    
#     # Compile the model
#     optimizer = Adam(learning_rate=0.0001)
#     model.compile(optimizer=optimizer, 
#                  loss='categorical_crossentropy', 
#                  metrics=['accuracy'])
    
#     return model

# # # Data augmentation configuration
# # datagen = ImageDataGenerator(
# #     rotation_range=10,
# #     width_shift_range=0.05,
# #     height_shift_range=0.05,
# #     horizontal_flip=True)
# # datagen.fit(X_train)

# # Federated Learning Main Loop
# global_model = None

# for round_num in range(NUM_ROUNDS):
#     print(f"\n=== Federated Learning Round {round_num + 1}/{NUM_ROUNDS} ===")
    
#     # 1. Initialize or copy the global model
#     if global_model is None:
#         print("Initializing global model...")
#         global_model = create_global_model()
#     else:
#         print("Using existing global model...")
    
#     # 2. Select clients
#     selected_client_indices = np.random.choice(
#         len(clients), 
#         size=int(NUM_CLIENTS * CLIENT_FRACTION), 
#         replace=False)
#     selected_clients = [clients[i] for i in selected_client_indices]
    
#     # Store client model weights
#     client_weights = []
    
#     # 3. Local training on clients
#     for client_idx, (client_X, client_y) in enumerate(selected_clients):
#         print(f"\nTraining Client {client_idx + 1}/{len(selected_clients)}...")
        
#         # Clone the global model
#         client_model = tf.keras.models.clone_model(global_model)
#         client_model.set_weights(global_model.get_weights())
        
#         # Compile the client model
#         client_model.compile(optimizer=Adam(learning_rate=0.0001),
#                            loss='categorical_crossentropy',
#                            metrics=['accuracy'])
        
#         # Local training
#         history = client_model.fit(
#             datagen.flow(client_X, client_y, batch_size=16),
#             epochs=5,
#             verbose=1)
        
#         # Save client weights
#         client_weights.append(client_model.get_weights())
    
#     # 4. Federated Averaging (FedAvg)
#     print("\nAggregating client model weights...")
#     global_weights = []
    
#     # Average weights layer by layer
#     for layer_idx in range(len(global_model.get_weights())):
#         layer_weights = np.array([weights[layer_idx] for weights in client_weights])
#         avg_layer_weights = np.mean(layer_weights, axis=0)
#         global_weights.append(avg_layer_weights)
    
#     # Update the global model
#     global_model.set_weights(global_weights)
    
#     # # 5. Evaluate the global model
#     # print("\nEvaluating the global model...")
#     # loss, accuracy = global_model.evaluate(X_test, y_test, verbose=0)
#     # print(f"Test Accuracy: {accuracy:.4f}, Loss: {loss:.4f}")

# # Save the final global model
# global_model.save('federated_vgg19_model2.h5')
# print("\nFederated learning completed. Model saved as federated_vgg19_model2.h5")

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import pandas as pd
# import os

# # Federated Learning Parameters
# NUM_CLASSES = 4  # glioma, meningioma, notumor, pituitary
# CLIENTS_PER_CLASS = 5  # 5 clients per class
# NUM_CLIENTS = NUM_CLASSES * CLIENTS_PER_CLASS  # 20 total clients
# NUM_ROUNDS = 2
# CLIENT_FRACTION = 0.5  # Fraction of clients participating each round

# # Early Stopping Parameters
# WINDOW_SIZE = 5  # Larger window for non-IID data
# MIN_DELTA = 0.001  # Smaller delta for non-IID
# PATIENCE = 5  # More patience for non-IID

# # Data Preparation (Assume X_train, y_train, X_test, y_test are prepared)
# # X_train shape: (n_samples, 200, 200, 3)
# # y_train shape: (n_samples, 4) - one-hot encoded

# # Split data into non-IID clients (each client gets only one class)
# def create_non_iid_clients(X_train, y_train):
#     # Convert one-hot to class indices
#     y_train_labels = np.argmax(y_train, axis=1)
    
#     # Separate data by class
#     class_data = {}
#     for class_id in range(NUM_CLASSES):
#         class_indices = np.where(y_train_labels == class_id)[0]
#         class_data[class_id] = X_train[class_indices], y_train[class_indices]
    
#     clients = []
#     # Distribute each class's data to CLIENTS_PER_CLASS clients
#     for class_id in range(NUM_CLASSES):
#         X_class, y_class = class_data[class_id]
#         samples_per_client = len(X_class) // CLIENTS_PER_CLASS
        
#         for client_id in range(CLIENTS_PER_CLASS):
#             start = client_id * samples_per_client
#             end = (client_id + 1) * samples_per_client
            
#             client_X = X_class[start:end]
#             client_y = y_class[start:end]
#             clients.append((client_X, client_y))
    
#     return clients

# clients = create_non_iid_clients(X_train, y_train)

# # Initialize the global model
# def create_global_model():
#     IMG_SIZE = (200, 200)
#     base_model = VGG19(
#         include_top=False,
#         input_shape=IMG_SIZE + (3,),
#         weights='imagenet')

#     # Freeze all except last 7 layers
#     for layer in base_model.layers:
#         layer.trainable = False
#     for layer in base_model.layers[-7:]:
#         layer.trainable = True

#     # Custom classification head
#     x = GlobalAveragePooling2D()(base_model.output)
#     x = Dropout(0.4)(x)
#     predict = Dense(NUM_CLASSES, activation='softmax')(x)

#     model = Model(inputs=base_model.input, outputs=predict)
    
#     # Compile with lower learning rate for non-IID
#     optimizer = Adam(learning_rate=0.00005)
#     model.compile(optimizer=optimizer, 
#                  loss='categorical_crossentropy', 
#                  metrics=['accuracy'])
    
#     return model

# # # Data augmentation configuration
# # datagen = ImageDataGenerator(
# #     rotation_range=10,
# #     width_shift_range=0.05,
# #     height_shift_range=0.05,
# #     horizontal_flip=True)

# # Create results directory
# os.makedirs('results_non_iid', exist_ok=True)

# # Initialize tracking variables
# results = []
# best_accuracy = 0.0
# best_model_weights = None
# recent_accuracies = []
# no_improvement_count = 0
# early_stop_triggered = False

# # Federated Learning Main Loop
# global_model = None

# for round_num in range(NUM_ROUNDS):
#     if early_stop_triggered:
#         print("Early stopping triggered. Ending training.")
#         break
        
#     print(f"\n=== Federated Learning Round {round_num + 1}/{NUM_ROUNDS} ===")
    
#     # 1. Initialize or copy the global model
#     if global_model is None:
#         print("Initializing global model...")
#         global_model = create_global_model()
#     else:
#         print("Using existing global model...")
    
#     # 2. Select clients (ensure balanced class participation)
#     selected_indices = []
#     for class_id in range(NUM_CLASSES):
#         class_client_indices = range(class_id*CLIENTS_PER_CLASS, (class_id+1)*CLIENTS_PER_CLASS)
#         selected = np.random.choice(
#             list(class_client_indices),
#             size=max(1, int(CLIENT_FRACTION * CLIENTS_PER_CLASS)),
#             replace=False)
#         selected_indices.extend(selected)
    
#     selected_clients = [clients[i] for i in selected_indices]
    
#     # Store client weights and sample sizes
#     client_weights = []
#     client_sizes = []
    
#     # 3. Local training on clients
#     for client_idx, (client_X, client_y) in enumerate(selected_clients):
#         client_class = np.argmax(client_y[0])  # Get class for this client
#         print(f"\nTraining Client {client_idx + 1}/{len(selected_clients)} (Class {client_class})...")
        
#         # Clone the global model
#         client_model = tf.keras.models.clone_model(global_model)
#         client_model.set_weights(global_model.get_weights())
        
#         # Compile with same parameters
#         client_model.compile(optimizer=Adam(learning_rate=0.00005),
#                            loss='categorical_crossentropy',
#                            metrics=['accuracy'])
        
#         # Local training with more epochs for non-IID
#         history = client_model.fit(
#             datagen.flow(client_X, client_y, batch_size=16),
#             epochs=5,  # Increased epochs for non-IID
#             verbose=1)
        
#         # Save client weights and data size
#         client_weights.append(client_model.get_weights())
#         client_sizes.append(len(client_X))
    
#     # 4. Weighted Federated Averaging
#     print("\nAggregating client model weights...")
#     global_weights = []
#     total_size = sum(client_sizes)
    
#     # Weighted average by client data size
#     for layer_idx in range(len(global_model.get_weights())):
#         weighted_sum = np.zeros_like(global_model.get_weights()[layer_idx])
#         for client_idx in range(len(client_weights)):
#             weighted_sum += client_weights[client_idx][layer_idx] * (client_sizes[client_idx] / total_size)
#         global_weights.append(weighted_sum)
    
#     global_model.set_weights(global_weights)
    
#     # 5. Evaluate the global model
#     print("\nEvaluating the global model...")
#     loss, accuracy = global_model.evaluate(X_test, y_test, verbose=0)
#     print(f"Test Accuracy: {accuracy:.4f}, Loss: {loss:.4f}")
    
#     # Record results
#     results.append({
#         'round': round_num + 1,
#         'accuracy': accuracy,
#         'loss': loss
#     })
    
#     # Check for best model
#     if accuracy > best_accuracy + MIN_DELTA:
#         best_accuracy = accuracy
#         best_model_weights = global_model.get_weights()
#         no_improvement_count = 0
#         print(f"New best accuracy: {best_accuracy:.4f}")
#     else:
#         no_improvement_count += 1
    
#     # Early stopping check
#     recent_accuracies.append(accuracy)
#     if len(recent_accuracies) >= WINDOW_SIZE:
#         window_avg = np.mean(recent_accuracies[-WINDOW_SIZE:])
#         prev_window_avg = np.mean(recent_accuracies[-WINDOW_SIZE-1:-1] if len(recent_accuracies) > WINDOW_SIZE else recent_accuracies[:-1])
        
#         if window_avg <= prev_window_avg + MIN_DELTA:
#             no_improvement_count += 1
#             print(f"No improvement count: {no_improvement_count}/{PATIENCE}")
#         else:
#             no_improvement_count = max(0, no_improvement_count - 1)
        
#         if no_improvement_count >= PATIENCE:
#             early_stop_triggered = True

# # Save results
# df_results = pd.DataFrame(results)
# df_results.to_excel('results_non_iid/federated_learning_results.xlsx', index=False)

# # Save best model
# if best_model_weights is not None:
#     global_model.set_weights(best_model_weights)
#     global_model.save('results_non_iid/best_global_model.h5')
#     print(f"Best model saved with accuracy: {best_accuracy:.4f}")

# # Save final model
# global_model.save('results_non_iid/final_global_model.h5')
# print("\nFederated learning completed. Final model saved.")

In [ ]:
# global_model = initialize_with_synthetic(X_synthetic, y_synthetic)

In [ ]:
# # Confusion Matrix Visualization
# from sklearn.metrics import classification_report,accuracy_score
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # Prepare data
# if len(y_test.shape) > 1 and y_test.shape[1] > 1:  # If one-hot encoded
#     y_test_labels = np.argmax(y_test, axis=1)
# else:
#     y_test_labels = y_test
    
# # Predict on the test set (using a small batch_size to prevent OOM)
# y_pred = global_model.predict(X_test, batch_size=8)
# y_pred_labels = np.argmax(y_pred, axis=1)

# # Define class names (adjust according to your dataset)
# class_names = ['glioma', 'meningioma', 'no_tumor', 'pituitary']

# # Print the full classification report
# print("="*65)
# print("Test Set Classification Report:")
# print("="*65)
# print(classification_report(y_test_labels, y_pred_labels, 
#                            target_names=class_names, digits=4))

# # Print overall accuracy
# test_loss, test_acc = global_model.evaluate(X_test, y_test, 
#                                            batch_size=8, verbose=0)
# print(f"\nOverall Test Set Accuracy: {test_acc:.4f}, Loss: {test_loss:.4f}")
# print("="*65)

In [ ]:
# import numpy as np
# from sklearn.metrics import classification_report
# from tensorflow.keras.models import load_model
# from tensorflow.keras.optimizers import Adam
# import os

# # 确保results目录存在
# os.makedirs('results621', exist_ok=True)

# # 1. 加载模型

# print("Loading model...")
# model = load_model('results621/best_global_model.h5')
# print("Model loaded successfully.\n")

# # 重新编译模型（使用原始训练时的相同参数）
# model.compile(optimizer=Adam(learning_rate=0.001),  # 使用与原训练相同的优化器
#               loss='categorical_crossentropy',     # 分类任务常用loss
#               metrics=['accuracy'])               # 监控准确率


# # 2. 准备数据
# if len(y_test.shape) > 1 and y_test.shape[1] > 1:  # 如果是one-hot编码
#     y_test_labels = np.argmax(y_test, axis=1)
# else:
#     y_test_labels = y_test

# # 3. 预测测试集（使用小batch_size防止内存不足）
# print("Making predictions...")
# y_pred = model.predict(X_test, batch_size=8)
# y_pred_labels = np.argmax(y_pred, axis=1)

# # 4. 定义类别名称（根据你的数据集调整）
# class_names = ['glioma', 'meningioma', 'no_tumor', 'pituitary']

# # 5. 计算并打印分类报告
# print("="*65)
# print("Generating classification report...")
# report = classification_report(y_test_labels, y_pred_labels, 
#                               target_names=class_names, digits=4)

# # 6. 计算整体准确率
# print("Evaluating model...")
# test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=8, verbose=0)

# # 7. 打印结果
# print("="*65)
# print("Test Set Classification Report:")
# print("="*65)
# print(report)
# print(f"\nOverall Test Set Accuracy: {test_acc:.4f}, Loss: {test_loss:.4f}")
# print("="*65)

# # 8. 将结果保存到文件
# report_path = 'results621/classification_report.txt'
# with open(report_path, 'w') as f:
#     f.write("="*65 + "\n")
#     f.write("Test Set Classification Report:\n")
#     f.write("="*65 + "\n")
#     f.write(report)
#     f.write(f"\nOverall Test Set Accuracy: {test_acc:.4f}, Loss: {test_loss:.4f}\n")
#     f.write("="*65 + "\n")

# print(f"\nClassification report saved to {report_path}")

In [ ]:
# # Confusion Matrix Visualization
# from sklearn.metrics import confusion_matrix
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # Prepare data
# if len(y_test.shape) > 1 and y_test.shape[1] > 1:  # If one-hot encoded
#     y_test_labels = np.argmax(y_test, axis=1)
# else:
#     y_test_labels = y_test

# y_pred = global_model.predict(X_test, batch_size=8)
# y_pred_labels = np.argmax(y_pred, axis=1)

# # Define class names (consistent with classification report)
# class_names = ['glioma', 'meningioma', 'no_tumor', 'pituitary']

# # Compute confusion matrix
# cm = confusion_matrix(y_test_labels, y_pred_labels)

# # Plot polished confusion matrix
# plt.figure(figsize=(10, 8))
# sns.set(font_scale=1.2)
# ax = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
#                  cbar=False, square=True,
#                  xticklabels=class_names, 
#                  yticklabels=class_names)

# # Enhance visualization
# ax.set(xlabel='Predicted Label', ylabel='True Label', 
#        title='Confusion Matrix')
# plt.xticks(rotation=45, ha='right')
# plt.yticks(rotation=0)
# plt.tight_layout()

# # Add colorbar on the right
# plt.colorbar(ax.collections[0], pad=0.15, label="Sample Count")

# # Save high-quality image (optional)
# plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
# plt.show()

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import confusion_matrix
# from tensorflow.keras.models import load_model
# from tensorflow.keras.optimizers import Adam
# import os

# # 确保results目录存在
# os.makedirs('results621', exist_ok=True)

# # 1. 加载模型
# print("Loading model...")
# model = load_model('results621/best_global_model.h5')

# # 2. 重新编译模型
# print("Compiling model...")
# model.compile(optimizer=Adam(learning_rate=0.001),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# # 3. 准备数据
# if len(y_test.shape) > 1 and y_test.shape[1] > 1:  # 如果是one-hot编码
#     y_test_labels = np.argmax(y_test, axis=1)
# else:
#     y_test_labels = y_test

# # 4. 预测测试集
# print("Making predictions...")
# y_pred = model.predict(X_test, batch_size=8)
# y_pred_labels = np.argmax(y_pred, axis=1)

# # 5. 定义类别名称
# class_names = ['glioma', 'meningioma', 'no_tumor', 'pituitary']

# # 6. 计算混淆矩阵
# print("Computing confusion matrix...")
# cm = confusion_matrix(y_test_labels, y_pred_labels)

# # 7. 绘制混淆矩阵
# plt.figure(figsize=(10, 8))
# sns.set(font_scale=1.2, style='whitegrid')  # 设置样式
# ax = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
#                 cbar=False, square=True,
#                 xticklabels=class_names,
#                 yticklabels=class_names)

# # 8. 美化图表
# ax.set(xlabel='Predicted Label', ylabel='True Label',
#       title='Confusion Matrix')
# plt.xticks(rotation=45, ha='right')
# plt.yticks(rotation=0)
# plt.tight_layout()

# # 9. 添加颜色条
# plt.colorbar(ax.collections[0], pad=0.15, label="Sample Count")

# # 10. 保存为PDF
# output_path = 'results621/confusion_matrix-15clients.pdf'
# plt.savefig(output_path,
#            format='pdf',
#            dpi=300,
#            bbox_inches='tight',
#            pad_inches=0.1,
#            transparent=False)

# print(f"\nConfusion matrix saved to {output_path}")
# plt.show()

In [ ]:
# from tensorflow.keras.applications.vgg19 import VGG19
# IMG_SIZE=(200,200)
# base_model = VGG19(
#     include_top=False,
#     input_shape=IMG_SIZE + (3,),
#     weights='imagenet')

# for layer in base_model.layers:
#     layer.trainable = True

In [ ]:
# base_model.summary()

In [ ]:
# # Customized layers
# x = GlobalAveragePooling2D()(base_model.output)
# x = Dropout(0.4)(x)
# predict = Dense(4,activation='softmax')(x)

# # create a model object
# model = Model(inputs = base_model.input,outputs = predict)
# # model.summary()

In [ ]:
# #compile our model.
# adam = Adam(learning_rate=0.0001)
# model.compile(optimizer=adam, loss = 'categorical_crossentropy', metrics=['accuracy'])

Optimizer Definition:
An instance of the Adam optimizer is defined here. Adam is a widely-used optimization algorithm in deep learning, known for its efficiency and computational effectiveness. It combines the benefits of both momentum and RMSProp, and adaptively adjusts the learning rate during training. By setting learning_rate=0.0001, a relatively small learning rate is specified, which can help the model converge more stably, albeit potentially requiring more epochs to reach optimal performance.

Model Compilation:
optimizer=adam: Specifies that the previously defined Adam optimizer should be used to perform backpropagation and weight updates during training.
loss='categorical_crossentropy': Chooses "categorical crossentropy" as the loss function. This is a commonly used loss function for multi-class classification problems, particularly when the output layer uses a softmax activation. Categorical crossentropy measures the difference between two probability distributions – in this case, the true distribution of labels versus the predicted probabilities by the model.

In [ ]:
# train_len = len(X_train)
# val_len = len(X_val)
# print("-----------Training Data length-----------------")
# print(train_len)

# print("-----------Validation Data length-----------------")
# print(val_len)

In [ ]:
# hist = model.fit(datagen.flow(X_train, y_train, batch_size=32),validation_data = (X_val,y_val),epochs =25,verbose=1)

### Plotting output results

In [ ]:
# #Plotting the results
# acc = hist.history['accuracy']
# val_acc = hist.history['val_accuracy']
# loss = hist.history['loss']
# val_loss = hist.history['val_loss']

In [ ]:
# plt.figure(figsize=(15,3))
# plt.subplot(1,2,1)
# plt.plot(acc,label='Training Accuracy')
# plt.plot(val_acc,label='Validation Accuracy')
# plt.legend(loc= "lower right")
# plt.title("Training and Validation Accuracy")

# plt.subplot(1,2,2)
# plt.plot(loss,label='Training Loss')
# plt.plot(val_loss,label='Validation Loss')
# plt.legend(loc= "upper right")
# plt.title("Training and Validation Loss")
# plt.savefig('VGGAccLoss.pdf', format='pdf')

In [ ]:
# from tensorflow.keras.models import load_model

In [ ]:
# #Saving the model
# model.save('bestvgg19-2.h5')

### Validation on Test set

##### Classification Report

In [ ]:
loss,acc = global_model.evaluate(X_test,y_test)

41/41 [==============================] - 2s 41ms/step - loss: 0.5823 - accuracy: 0.8352


In [ ]:
# from sklearn.metrics import classification_report,accuracy_score
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# predicted_classes = np.argmax(global_model.predict(X_test), axis = 1)
# print(classification_report(np.argmax(y_test,axis=1), predicted_classes,target_names=['glioma','meningioma','no_tumor','pituitary']))

##### Confusion Matrix

In [ ]:
# import itertools
# # pred_Y = global_model.predict(X_test, batch_size = 8, verbose = True)

# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     target_names=['glioma','meningioma','no_tumor','pituitary']

#     if target_names is not None:
#         tick_marks = np.arange(len(target_names))
#         plt.xticks(tick_marks, target_names, rotation=45)
#         plt.yticks(tick_marks, target_names)
    
#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

# # Predict the values from the validation dataset
# Y_pred = global_model.predict(X_test, batch_size=8)
# # Convert predictions classes to one hot vectors 
# Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# # Convert validation observations to one hot vectors
# # compute the confusion matrix
# rounded_labels=np.argmax(y_test, axis=1)
# confusion_mtx = confusion_matrix(rounded_labels, Y_pred_classes)

 

# # plot the confusion matrix
# plot_confusion_matrix(confusion_mtx, classes = range(4)) 
# plt.savefig('VGGCM.pdf', format='pdf')

In [ ]:
# # 使用Seaborn的改进版本（无十字线+更美观）
# import seaborn as sns

# # 预测
# y_pred = global_model.predict(X_test, batch_size=8)
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_true = np.argmax(y_test, axis=1)

# # 混淆矩阵
# cm = confusion_matrix(y_true, y_pred_classes)
# class_names = ['glioma', 'meningioma', 'no_tumor', 'pituitary']

# # 绘图
# plt.figure(figsize=(10,8))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
#             xticklabels=class_names,
#             yticklabels=class_names,
#             cbar=False,
#             linewidths=0.5,
#             linecolor='lightgray')
# plt.title('Confusion Matrix (No Tumor)', pad=20)
# plt.xticks(rotation=45, ha='right')
# plt.yticks(rotation=0)
# plt.xlabel('Predicted Label', labelpad=10)
# plt.ylabel('True Label', labelpad=10)
# plt.tight_layout()
# plt.savefig('Confusion_Matrix.pdf', bbox_inches='tight')
# plt.show()

#### Plotting sample predictions

In [ ]:
# y_hat = model.predict(X_test)

# # define text labels 
# target_labels = ['glioma','meningioma','no_tumor','pituitary']

# # plot a random sample of test images, their predicted labels, and ground truth
# fig = plt.figure(figsize=(20, 8))
# for i, idx in enumerate(np.random.choice(X_test.shape[0], size=12, replace=False)):
#     ax = fig.add_subplot(4,4, i+1, xticks=[], yticks=[])
#     ax.imshow(np.squeeze(X_test[idx]))
#     pred_idx = np.argmax(y_hat[idx])
#     true_idx = np.argmax(y_test[idx])
#     ax.set_title("{} ({})".format(target_labels[pred_idx], target_labels[true_idx]),
#                  color=("blue" if pred_idx == true_idx else "orange"))

In [ ]:
# model = load_model('bestvgg19-2.h5')

In [ ]:
# import seaborn as sns
# import pandas as pd
# from sklearn.datasets import make_classification
# from sklearn.preprocessing import label_binarize
# #from scipy import interp
# from itertools import cycle
# import pandas as pd
# %matplotlib inline
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.metrics import roc_curve, auc

In [ ]:
# y_test = np.array(y_test)

# n_classes = 4

# pred_Y = model.predict(X_test, batch_size = 16, verbose = True)
# # Plot linewidth.
# lw = 2

# # Compute ROC curve and ROC area for each class


# # Compute ROC curve and ROC area for each class
# fpr = dict()
# tpr = dict()
# roc_auc = dict()
# for i in range(n_classes):
#     fpr[i], tpr[i], _ = roc_curve(y_test[:, i], pred_Y[:, i])
#     roc_auc[i] = auc(fpr[i], tpr[i])
#     # Compute micro-average ROC curve and ROC area
# fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), pred_Y.ravel())
# roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
# # Plot of a ROC curve for a specific class
# for i in range(n_classes):
#     plt.figure()
#     plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
#     plt.plot([0, 1], [0, 1], 'k--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver operating characteristic example')
#     plt.legend(loc="lower right")
#     plt.show()

# # First aggregate all false positive rates
# all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# # Then interpolate all ROC curves at this points
# mean_tpr = np.zeros_like(all_fpr)
# for i in range(n_classes):
#     mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# # Finally average it and compute AUC
# mean_tpr /= n_classes

# fpr["macro"] = all_fpr
# tpr["macro"] = mean_tpr
# roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# # Plot all ROC curves
# fig = plt.figure(figsize=(12, 8))
# plt.plot(fpr["micro"], tpr["micro"],
#          label='micro-average ROC curve (area = {0:0.2f})'
#                ''.format(roc_auc["micro"]),
#          color='deeppink', linestyle=':', linewidth=4)

# plt.plot(fpr["macro"], tpr["macro"],
#          label='macro-average ROC curve (area = {0:0.2f})'
#                ''.format(roc_auc["macro"]),
#          color='navy', linestyle=':', linewidth=4)

# colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
# for i, color in zip(range(n_classes), colors):
#     plt.plot(fpr[i], tpr[i], color=color, lw=lw,
#              label='ROC curve of class {0} (area = {1:0.2f})'
#              ''.format(i, roc_auc[i]))


# plt.plot([0, 1], [0, 1], 'k--', lw=lw)
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Some extension of Receiver operating characteristic to multi-class')
# plt.legend(loc="lower right")
# sns.despine()
# plt.show()

Count Prediction time

In [ ]:
# from tensorflow.keras.models import load_model
# model = load_model('bestvgg19.h5')

In [ ]:
# import os
# import time
# from tensorflow.keras.preprocessing import image
# import numpy as np

# # Define the labels
# labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

# # Define the directories
# directories = {
#     'glioma': './Dataset/Testingprediction/glioma',
#     'meningioma': './Dataset/Testingprediction/meningioma',
#     'notumor': './Dataset/Testingprediction/notumor',
#     'pituitary': './Dataset/Testingprediction/pituitary'
# }

# # Function to measure prediction time for images in a directory
# def measure_prediction_time(model, directory):
#     times = []
#     for img_name in os.listdir(directory):
#         img_path = os.path.join(directory, img_name)
#         img = image.load_img(img_path, target_size=(200, 200))
#         x = image.img_to_array(img)
#         x = x / 255.0
#         x = np.expand_dims(x, axis=0)
        
#         start_time = time.time()
#         predict_image = model.predict(x)
#         end_time = time.time()
        
#         prediction_time = end_time - start_time
#         times.append(prediction_time)
    
#     return times

# # Dictionary to store times for each category
# prediction_times = {}

# # Measure prediction times for each category
# for label, directory in directories.items():
#     times = measure_prediction_time(model, directory)
#     prediction_times[label] = times
#     avg_time = np.mean(times)
#     max_time = np.max(times)
#     min_time = np.min(times)
#     print(f"{label}: Average time = {avg_time:.4f} seconds, Max time = {max_time:.4f} seconds, Min time = {min_time:.4f} seconds")

# # If you want to print the overall results for all categories
# overall_times = [time for times in prediction_times.values() for time in times]
# avg_time = np.mean(overall_times)
# max_time = np.max(overall_times)
# min_time = np.min(overall_times)
# print(f"Overall: Average time = {avg_time:.4f} seconds, Max time = {max_time:.4f} seconds, Min time = {min_time:.4f} seconds")


In [ ]:
# import os
# import time
# import numpy as np
# import pandas as pd
# from tensorflow.keras.preprocessing import image

# # Define the labels
# labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

# # Define the directories
# directories = {
#     'glioma': './Dataset/Testingprediction/glioma',
#     'meningioma': './Dataset/Testingprediction/meningioma',
#     'notumor': './Dataset/Testingprediction/notumor',
#     'pituitary': './Dataset/Testingprediction/pituitary'
# }

# # Function to measure prediction time for images in a directory
# def measure_prediction_time(model, directory):
#     times = []
#     for img_name in os.listdir(directory):
#         img_path = os.path.join(directory, img_name)
#         img = image.load_img(img_path, target_size=(200, 200))
#         x = image.img_to_array(img)
#         x = x / 255.0
#         x = np.expand_dims(x, axis=0)
        
#         start_time = time.time()
#         predict_image = model.predict(x)
#         end_time = time.time()
        
#         prediction_time = end_time - start_time
#         times.append(prediction_time)
    
#     return times

# # Dictionary to store times for each category
# prediction_times = {}

# # Measure prediction times for each category
# for label, directory in directories.items():
#     times = measure_prediction_time(model, directory)
#     prediction_times[label] = times

# # Create a DataFrame to organize the results
# results = {
#     'Category': [],
#     'Average Time (s)': [],
#     'Max Time (s)': [],
#     'Min Time (s)': []
# }

# for label, times in prediction_times.items():
#     avg_time = np.mean(times)
#     max_time = np.max(times)
#     min_time = np.min(times)
#     results['Category'].append(label)
#     results['Average Time (s)'].append(avg_time)
#     results['Max Time (s)'].append(max_time)
#     results['Min Time (s)'].append(min_time)

# # Add overall statistics
# overall_times = [time for times in prediction_times.values() for time in times]
# avg_time = np.mean(overall_times)
# max_time = np.max(overall_times)
# min_time = np.min(overall_times)
# results['Category'].append('Overall')
# results['Average Time (s)'].append(avg_time)
# results['Max Time (s)'].append(max_time)
# results['Min Time (s)'].append(min_time)

# # Create a DataFrame and print it
# df = pd.DataFrame(results)
# print(df)
